#Experiment

## Setup

In [1]:
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_excel('/content/drive/MyDrive/data/idioms/corpus_complete.xlsx')
df.head()

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
0,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
1,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...",general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
2,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
3,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
4,A mother sheep with two black baby sheep.,Мать-овца с двумя черными овцами.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN


In [4]:
trans_df = df[['context_en', 'context_ru']].rename(columns={'context_en': 'source', 'context_ru': 'target'})
trans_df.tail()

,source,target
1529,You reap what you sow.,"Что посеешь, то и пожнёшь."
1530,They give and reap what they sow.,"Дающий получит, посеявший пожнет."
1531,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнёшь — никому не отдадут ..."
1532,He reaps what he sows.,"Он пожнёт то, что и посеял."
1533,So the next time you feel annoyed when you hea...,"Так что, когда в следующий раз вы почувствуете..."


## Yandex Translator

The code for translatin via Yandex Translator service is not listed here as it requires a special key for its API usage. Thus, these translations were obtained with the help of my scientific supervisor

## Google Translator

### Setup

In [ ]:
!pip install googletrans==3.1.0a0 -q

In [ ]:
from googletrans import Translator

### Translation

In [28]:
g_translator = Translator()
g_trans = []

for context in trans_df['source']:
    g_trans.append(g_translator.translate(context, dest='ru', src='en').text)

g_trans[-5:]

['Что посеешь то и пожнешь.',
 'Они отдают и пожинают то, что сеют.',
 'Что посеешь, то и пожнешь — никому не будет отдано предпочтение.',
 'Он пожинает то, что сеет.',
 'Так что в следующий раз, когда вы почувствуете раздражение, услышав, как ваша вторая половинка нежится в теплой ванне и беспрестанно напевает одну и ту же оперную арию, будьте благодарны судьбе.']

In [32]:
with open('/content/drive/MyDrive/data/idioms/g_trans.txt', 'w', encoding='utf-8') as g_trans_save:
    g_trans_save.write('\n'.join(g_trans))

## Hugging Face Models

## Setup

In [6]:
!pip install transformers -q
!pip install sacremoses -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.8 MB/s eta 0:00:00


In [11]:
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm

In [10]:
class ContextDataset(Dataset):
    def __init__(self, contexts):
        self._data = contexts

    def __len__(self):
        return len(self._data)

    def __getitem__(self, index: int):
        return self._data[index]

In [ ]:
device = 'cuda'

### Opus MT

#### Setup

In [19]:
from transformers import MarianMTModel, MarianTokenizer

In [20]:
class OPUSPipeline:
    def __init__(
            self,
            model,
            tokenizer,
            dataset: ContextDataset,
            max_length: int,
            batch_size: int,
            device: str
    ):
        self._model = model
        self._tokenizer = tokenizer
        self._dataset = dataset
        self._device = device
        self._max_length = max_length
        self._batch_size = batch_size
        self._model.to(device)

    def infer(self):
        dataloader = DataLoader(self._dataset, batch_size=self._batch_size)
        predictions = []

        for batch in tqdm(dataloader):
            predictions.extend(self._infer_batch(batch))

        return predictions

    def _infer_batch(self, batch):
        inputs = self._tokenizer(batch,
                                 padding=True, truncation=True,
                                 max_length=self._max_length, return_tensors="pt").to(self._device)
        outputs = self._model.generate(**inputs, max_length=self._max_length)
        return [self._tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

#### Translation

In [21]:
opus_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
opus_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [22]:
dataset = ContextDataset(trans_df['source'])
pipeline = OPUSPipeline(opus_model, opus_tokenizer, dataset, 120, 16, device)

In [23]:
translations = pipeline.infer()

  0%|          | 0/96 [00:00<?, ?it/s]

In [24]:
translations[:5]

['Столовое стадо чёрных овец выпасается вблизи берега',
 'Женщина, изгибающаяся по рельсам, глядя на белую овцу и чёрную овцу.',
 'Овечка, стоящая рядом с маленькой чёрной овечкой.',
 'Большая белая овца и две маленькие чёрные овцы в амбаре',
 'Мать-овца с двумя черными детьми-овцами.']

In [25]:
with open('/content/drive/MyDrive/data/idioms/opus_trans.txt', 'w', encoding='utf-8') as opus_trans_save:
    opus_trans_save.write('\n'.join(translations))

### mBART

#### Setup

In [8]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from tqdm import tqdm

In [12]:
class MBARTPipeline:
    def __init__(
            self,
            model,
            tokenizer,
            dataset: ContextDataset,
            max_length: int,
            batch_size: int,
            device: str
    ):
        self._model = model
        self._tokenizer = tokenizer
        self._dataset = dataset
        self._device = device
        self._max_length = max_length
        self._batch_size = batch_size
        self._model.to(device)

    def infer(self):
        dataloader = DataLoader(self._dataset, batch_size=self._batch_size)
        predictions = []

        for batch in tqdm(dataloader):
            predictions.extend(self._infer_batch(batch))

        return predictions

    def _infer_batch(self, batch):
        inputs = self._tokenizer(batch,
                                 padding=True, truncation=True,
                                 max_length=self._max_length, return_tensors="pt").to(self._device)
        outputs = self._model.generate(**inputs, max_length=self._max_length, forced_bos_token_id=self._tokenizer.lang_code_to_id["ru_RU"])
        return [trans for trans in self._tokenizer.batch_decode(outputs, skip_special_tokens=True)]

#### Translation

In [13]:
mbart_tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
mbart_tokenizer.src_lang = 'en_XX'
mbart_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tqdm.pandas()

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [14]:
dataset = ContextDataset(trans_df['source'])
pipeline = MBARTPipeline(mbart_model, mbart_tokenizer, dataset, 120, 8, device)

In [15]:
translations = pipeline.infer()

  0%|          | 0/192 [00:00<?, ?it/s]

In [18]:
translations[:5]

['Коровь чёрных овец, живущих рядом с берегом',
 'женщина, склоняясь над железной дорогой, смотрит на белую овцу и черную овцу',
 'Овец, стоящий рядом с маленькой черной овкой.',
 'Большая белая овца и две маленькие черные овцы в barn',
 'Мать овцы с двумя черными детьми овц.']

In [17]:
with open('/content/drive/MyDrive/data/idioms/mbart_trans.txt', 'w', encoding='utf-8') as mbart_trans_save:
    mbart_trans_save.write('\n'.join(translations))

## Evaluation

### Setup

In [1]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00


In [4]:
from evaluate import load
from tqdm.notebook import tqdm

#### Corpus Preparation

In [5]:
from google.colab import drive
import pandas as pd

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_excel('/content/drive/MyDrive/data/idioms/corpus_complete.xlsx')
df.head()

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
0,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
1,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...",general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
2,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
3,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
4,A mother sheep with two black baby sheep.,Мать-овца с двумя черными овцами.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN


In [8]:
trans_df = df[['context_en', 'context_ru']].rename(columns={'context_en': 'source', 'context_ru': 'target'})
trans_df.tail()

,source,target
1529,You reap what you sow.,"Что посеешь, то и пожнёшь."
1530,They give and reap what they sow.,"Дающий получит, посеявший пожнет."
1531,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнёшь — никому не отдадут ..."
1532,He reaps what he sows.,"Он пожнёт то, что и посеял."
1533,So the next time you feel annoyed when you hea...,"Так что, когда в следующий раз вы почувствуете..."


#### Translations Loading

In [9]:
translations = {}

In [10]:
# @title Yandex Translations {display-mode: "form"}

with open('/content/drive/MyDrive/data/idioms/ya_trans.txt', 'r', encoding='utf-8') as ya_file:
    ya_trans = ya_file.read()

translations['yandex'] = ya_trans.split('\n')
translations['yandex'] = translations['yandex'][:-1] #delete this annoying empty line
translations['yandex'][:5]

['Стадо черных овец пасется недалеко от берега',
 'дама, перегнувшаяся через перила и смотрящая на белую и черную овцу',
 'Овца, стоящая рядом с детенышем черной овцы.',
 'Большая белая овца и два детеныша черной овцы в сарае',
 'Овца-мать с двумя черными овечьими детенышами.']

In [11]:
# @title Google Translations {display-mode: "form"}

with open('/content/drive/MyDrive/data/idioms/g_trans.txt', 'r', encoding='utf-8') as g_file:
    g_trans = g_file.read()

translations['google'] = g_trans.split('\n')
translations['google'][:5]

['Стадо черных овец пасется у берега',
 'дама, склонившаяся над перилами, смотрит на белую и черную овцу',
 'Овца стоит рядом с детенышем черной овцы.',
 'Большая белая овца и две черные овцы в сарае',
 'Мать-овца с двумя черными овечками.']

In [12]:
# @title OPUS-MT Translations {display-mode: "form"}

with open('/content/drive/MyDrive/data/idioms/opus_trans.txt', 'r', encoding='utf-8') as opus_file:
    opus_trans = opus_file.read()

translations['opus'] = opus_trans.split('\n')
translations['opus'][:5]

['Столовое стадо чёрных овец выпасается вблизи берега',
 'Женщина, изгибающаяся по рельсам, глядя на белую овцу и чёрную овцу.',
 'Овечка, стоящая рядом с маленькой чёрной овечкой.',
 'Большая белая овца и две маленькие чёрные овцы в амбаре',
 'Мать-овца с двумя черными детьми-овцами.']

In [13]:
# @title mBART Translations {display-mode: "form"}

with open('/content/drive/MyDrive/data/idioms/mbart_trans.txt', 'r', encoding='utf-8') as mbart_file:
    mbart_trans = mbart_file.read()

translations['mbart'] = mbart_trans.split('\n')
translations['mbart'][:5]

['Коровь чёрных овец, живущих рядом с берегом',
 'женщина, склоняясь над железной дорогой, смотрит на белую овцу и черную овцу',
 'Овец, стоящий рядом с маленькой черной овкой.',
 'Большая белая овца и две маленькие черные овцы в barn',
 'Мать овцы с двумя черными детьми овц.']

In [14]:
for system, trans in translations.items():
    trans_df[system] = trans
trans_df

,source,target,yandex,google,opus,mbart
0,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,Стадо черных овец пасется недалеко от берега,Стадо черных овец пасется у берега,Столовое стадо чёрных овец выпасается вблизи б...,"Коровь чёрных овец, живущих рядом с берегом"
1,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...","дама, перегнувшаяся через перила и смотрящая н...","дама, склонившаяся над перилами, смотрит на бе...","Женщина, изгибающаяся по рельсам, глядя на бел...","женщина, склоняясь над железной дорогой, смотр..."
2,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,"Овца, стоящая рядом с детенышем черной овцы.",Овца стоит рядом с детенышем черной овцы.,"Овечка, стоящая рядом с маленькой чёрной овечкой.","Овец, стоящий рядом с маленькой черной овкой."
3,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,Большая белая овца и два детеныша черной овцы ...,Большая белая овца и две черные овцы в сарае,Большая белая овца и две маленькие чёрные овцы...,Большая белая овца и две маленькие черные овцы...
4,A mother sheep with two black baby sheep.,Мать-овца с двумя черными овцами.,Овца-мать с двумя черными овечьими детенышами.,Мать-овца с двумя черными овечками.,Мать-овца с двумя черными детьми-овцами.,Мать овцы с двумя черными детьми овц.
...,...,...,...,...,...,...
1529,You reap what you sow.,"Что посеешь, то и пожнёшь.","Вы пожинаете то, что посеяли.",Что посеешь то и пожнешь.,"Ты пожинаешь то, что сеешь.","Ты рождёшь то, что сеешь."
1530,They give and reap what they sow.,"Дающий получит, посеявший пожнет.","Они дают и пожинают то, что посеяли.","Они отдают и пожинают то, что сеют.","Они дают и пожинают то, что сеют.","Они дают и собирают то, что сеют."
1531,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнёшь — никому не отдадут ...","Что посеешь, то и пожнешь — никому не будет от...","Что посеешь, то и пожнешь — никому не будет от...","То, что ты сеешь, ты пожинаешь — никому не буд...","Что ты сеешь, то и собираешь — никому не дадут..."
1532,He reaps what he sows.,"Он пожнёт то, что и посеял.","Он пожинает то, что сеет.","Он пожинает то, что сеет.","Он пожинает то, что сеет.","Он выращивает то, что сает."


In [18]:
def replace_yo(text):
    return text.replace('ё', 'е')

In [19]:
trans_df = trans_df.applymap(replace_yo)

In [20]:
trans_df.tail()

,source,target,yandex,google,opus,mbart
1529,You reap what you sow.,"Что посеешь, то и пожнешь.","Вы пожинаете то, что посеяли.",Что посеешь то и пожнешь.,"Ты пожинаешь то, что сеешь.","Ты рождешь то, что сеешь."
1530,They give and reap what they sow.,"Дающий получит, посеявший пожнет.","Они дают и пожинают то, что посеяли.","Они отдают и пожинают то, что сеют.","Они дают и пожинают то, что сеют.","Они дают и собирают то, что сеют."
1531,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнешь — никому не отдадут ...","Что посеешь, то и пожнешь — никому не будет от...","Что посеешь, то и пожнешь — никому не будет от...","То, что ты сеешь, ты пожинаешь — никому не буд...","Что ты сеешь, то и собираешь — никому не дадут..."
1532,He reaps what he sows.,"Он пожнет то, что и посеял.","Он пожинает то, что сеет.","Он пожинает то, что сеет.","Он пожинает то, что сеет.","Он выращивает то, что сает."
1533,So the next time you feel annoyed when you hea...,"Так что, когда в следующий раз вы почувствуете...","Так что в следующий раз, когда вы почувствуете...","Так что в следующий раз, когда вы почувствуете...","Так что в следующий раз, когда ты почувствуешь...","Так что в следующий раз, когда вы чувствуете р..."


#### BLEURT Metric

##### Setup

In [26]:
!pip install git+https://github.com/google-research/bleurt.git -q

  Preparing metadata (setup.py) ... done


In [27]:
from statistics import mean

In [28]:
bleurt_scores = {}
bleurt = load('bleurt', module_type='metric', checkpoint="BLEURT-20")

##### Evaluation

In [51]:
for system in tqdm(translations.keys()):
    scores = bleurt.compute(predictions=trans_df[system], references=trans_df['target'])
    bleurt_scores[system] = mean(list(scores.values())[0])

  0%|          | 0/4 [00:00<?, ?it/s]

system	score


ValueError: too many values to unpack (expected 2)

In [52]:
print('system\tscore')
for system, score in bleurt_scores.items():
    print(f'{system} - {score}')

system	score
yandex - 0.4994591271293482
google - 0.4848739783215243
opus - 0.33244475167890647
mbart - 0.28155515786148655


#### COMET Metric

##### Setup

In [20]:
!pip install unbabel-comet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [21]:
comet_scores = {}
comet = load('comet')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


##### Evaluation

In [22]:
for system in tqdm(translations.keys()):
    scores = comet.compute(predictions=trans_df[system], references=trans_df['target'], sources=trans_df['source'], progress_bar=True)
    comet_scores[system] = scores['mean_score']

  0%|          | 0/4 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Predicting DataLoader 0: 100%|██████████| 96/96 [00:36<00:00,  2.66it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:

In [23]:
print('system\tscore')
for system, score in comet_scores.items():
    print(f'{system} - {score}')

system	score
yandex - 0.8814336031558772
google - 0.8714475046158459
opus - 0.802090107462086
mbart - 0.7732720607223846


#### LitTER Metric (Custom Implementation)

#####Setup

In [2]:
!pip install word2word -q

In [31]:
from word2word import Word2word
import re
from statistics import mean

In [15]:
en2ru = Word2word('en', 'ru')

In [56]:
class CustomLitTER:
    def __init__(self, dictionary: Word2word):
        self._dictionary = dictionary


    def evaluate(self, target_texts: pd.Series, translations: pd.Series, idioms: pd.Series) -> dict:
        idiom_counts = idioms.value_counts()
        idiom_err_counts = {idiom: 0 for idiom in idioms.unique()}
        trans_errors = []
        for target, translation, idiom in tqdm(zip(target_texts, translations, idioms)):
            if self._check_translation(idiom, target, translation) is False:
                idiom_err_counts[idiom] += 1
                trans_errors.append(1)
                continue
            trans_errors.append(0)

        idiom_err_proportion = {idiom: round(errors / idiom_counts[idiom], 2) for idiom, errors in idiom_err_counts.items()}
        score = mean(trans_errors)

        return {'per_idiom': idiom_err_proportion, 'mean_score': score}


    def _check_translation(self, idiom: str, target: str, translation: str) -> bool:
        idiom_tokens = self._tokenize(idiom)
        target_tokens = self._tokenize(target)
        trans_tokens = self._tokenize(translation)

        blocklist = self._create_blocklist(idiom_tokens)
        blocklist = self._sort_blocklist(blocklist, target_tokens)

        for word in trans_tokens:
            if word in blocklist:
                return False
        return True

    def _create_blocklist(self, idiom_words: list) -> list:
        blocklist = []
        for word in idiom_words:
            try:
                blocklist.extend(self._dictionary(word, 1_000_000_000))
            except KeyError:
                print(f'word {word} is not in the dictionary.')
                continue
        return blocklist

    @staticmethod
    def _sort_blocklist(blocklist: list, target_words: list):
        for word in target_words:
            if word in blocklist:
                blocklist.remove(word)
        return blocklist

    @staticmethod
    def _tokenize(text: str) -> list:
        return re.findall(r'\w+', text)

In [39]:
refs_df = df[['context_ru', 'idiom_en']].rename(columns={'context_ru': 'target', 'idiom_en': 'idiom'})
refs_df.tail()

,target,idiom
1529,"Что посеешь, то и пожнёшь.",you reap what you sow
1530,"Дающий получит, посеявший пожнет.",you reap what you sow
1531,"Что посеешь, то и пожнёшь — никому не отдадут ...",you reap what you sow
1532,"Он пожнёт то, что и посеял.",you reap what you sow
1533,"Так что, когда в следующий раз вы почувствуете...",your better half


In [38]:
trans_df = pd.DataFrame(translations)
trans_df.head()

,yandex,google,opus,mbart
0,Стадо черных овец пасется недалеко от берега,Стадо черных овец пасется у берега,Столовое стадо чёрных овец выпасается вблизи б...,"Коровь чёрных овец, живущих рядом с берегом"
1,"дама, перегнувшаяся через перила и смотрящая н...","дама, склонившаяся над перилами, смотрит на бе...","Женщина, изгибающаяся по рельсам, глядя на бел...","женщина, склоняясь над железной дорогой, смотр..."
2,"Овца, стоящая рядом с детенышем черной овцы.",Овца стоит рядом с детенышем черной овцы.,"Овечка, стоящая рядом с маленькой чёрной овечкой.","Овец, стоящий рядом с маленькой черной овкой."
3,Большая белая овца и два детеныша черной овцы ...,Большая белая овца и две черные овцы в сарае,Большая белая овца и две маленькие чёрные овцы...,Большая белая овца и две маленькие черные овцы...
4,Овца-мать с двумя черными овечьими детенышами.,Мать-овца с двумя черными овечками.,Мать-овца с двумя черными детьми-овцами.,Мать овцы с двумя черными детьми овц.


#####Evaluation on full corpus

In [ ]:
litter = CustomLitTER(en2ru)

Yandex

In [ ]:
ya_litter_score = litter.evaluate(refs_df['target'], trans_df['yandex'], refs_df['idiom'])
ya_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.3272490221642764

Google

In [ ]:
g_litter_score = litter.evaluate(refs_df['target'], trans_df['google'], refs_df['idiom'])
g_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.36310299869621904

OPUS

In [ ]:
opus_litter_score = litter.evaluate(refs_df['target'], trans_df['opus'], refs_df['idiom'])
opus_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.5045632333767927

mBART

In [ ]:
mbart_litter_score = litter.evaluate(refs_df['target'], trans_df['mbart'], refs_df['idiom'])
mbart_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.5136897001303781

In [ ]:
err_rate_df = pd.DataFrame(index=refs_df['idiom'].unique())
err_rate_df['yandex'] = ya_litter_score['per_idiom'].values()
err_rate_df['google'] = g_litter_score['per_idiom'].values()
err_rate_df['OPUS'] = opus_litter_score['per_idiom'].values()
err_rate_df['mBART'] = ya_litter_score['per_idiom'].values()
err_rate_df

,yandex,google,OPUS,mBART
a black sheep,0.30,0.30,0.30,0.30
a breath of fresh air,0.00,0.00,1.00,0.00
a dark horse,0.80,0.70,0.70,0.80
a fact of life,0.80,1.00,1.00,0.80
a fair-weather friend,0.75,1.00,0.25,0.75
...,...,...,...,...
when push comes to shove,1.00,1.00,1.00,1.00
work like a charm,0.00,1.00,1.00,0.00
wring your hands,1.00,1.00,1.00,1.00
you reap what you sow,0.83,0.83,0.50,0.83


In [ ]:
err_rate_df.to_csv('/content/drive/MyDrive/data/idioms/error_rates.csv')

System | Score
--- | ---
Yandex | 0.33
Google | 0.36
OPUS | 0.51
mBART | 0.51

#####Evaluation on idiomatic contexts

In [82]:
refs_df = df[['context_ru', 'idiom_en', 'is_idiomatic']].rename(columns={'context_ru': 'target', 'idiom_en': 'idiom'})
refs_df = refs_df.loc[(refs_df['is_idiomatic'] == True) | (refs_df['is_idiomatic'] == 'Ambiguous')]
refs_df

,target,idiom,is_idiomatic
5,"Я всем нравлюсь, потому что я тихая и послушна...",a black sheep,True
6,"Как художник со свободным духом, я всегда был ...",a black sheep,True
7,Дядя Фриц был паршивой овцой в нашей семье; мы...,a black sheep,True
8,"Мой дядя был паршивой овцой в семье, и мы стар...",a black sheep,True
9,"Дебби — паршивая овца в семье, она ушла из дом...",a black sheep,True
...,...,...,...
1529,"Что посеешь, то и пожнёшь.",you reap what you sow,Ambiguous
1530,"Дающий получит, посеявший пожнет.",you reap what you sow,True
1531,"Что посеешь, то и пожнёшь — никому не отдадут ...",you reap what you sow,True
1532,"Он пожнёт то, что и посеял.",you reap what you sow,True


In [83]:
trans_df = pd.DataFrame(translations)
trans_df = trans_df.iloc[refs_df.index]
trans_df

,yandex,google,opus,mbart
5,"Я всем нравлюсь, потому что я такой тихий и по...","Я всем нравлюсь, потому что я такой тихий и по...","Я всем нравлюсь, потому что я такая тихая и по...","Всем мне нравится, потому что я так тихий и по..."
6,"Как художник со свободным духом, я всегда был ...","Как свободолюбивый художник, я всегда был бело...","Будучи свободным духовным художником, я всегда...","Будучи свободным художником, я всегда была чер..."
7,Дядя Фриц был белой вороной в нашей семье; мы ...,Дядя Фриц был белой вороной в семье; мы всегда...,Дядя Фриц был черной овцой семьи; мы всегда ду...,"Мы всегда думали, что он эмигрировал в Аргенти..."
8,"Мой дядя был белой вороной в семье, и нас нико...","Мой дядя был паршивой овцой в семье, и нас ник...","Мой дядя был чёрной овцой семьи, и нас никогда...",Мой дядя был черным овецом семьи и мы никогда ...
9,"Дебби - белая ворона в семье, она ушла из дома...",Дебби — паршивая овца в семье: в семнадцать ле...,"Дебби - черная овца семьи, которая ушла из дом...","Дебби — черная овца семьи, которая покинула до..."
...,...,...,...,...
1529,"Вы пожинаете то, что посеяли.",Что посеешь то и пожнешь.,"Ты пожинаешь то, что сеешь.","Ты рождёшь то, что сеешь."
1530,"Они дают и пожинают то, что посеяли.","Они отдают и пожинают то, что сеют.","Они дают и пожинают то, что сеют.","Они дают и собирают то, что сеют."
1531,"Что посеешь, то и пожнешь — никому не будет от...","Что посеешь, то и пожнешь — никому не будет от...","То, что ты сеешь, ты пожинаешь — никому не буд...","Что ты сеешь, то и собираешь — никому не дадут..."
1532,"Он пожинает то, что сеет.","Он пожинает то, что сеет.","Он пожинает то, что сеет.","Он выращивает то, что сает."


In [84]:
litter = CustomLitTER(en2ru)

Yandex

In [85]:
ya_litter_score = litter.evaluate(refs_df['target'], trans_df['yandex'], refs_df['idiom'])
ya_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.41284403669724773

Google

In [86]:
g_litter_score = litter.evaluate(refs_df['target'], trans_df['google'], refs_df['idiom'])
g_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.4873853211009174

OPUS

In [87]:
opus_litter_score = litter.evaluate(refs_df['target'], trans_df['opus'], refs_df['idiom'])
opus_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.6467889908256881

mBART

In [88]:
mbart_litter_score = litter.evaluate(refs_df['target'], trans_df['mbart'], refs_df['idiom'])
mbart_litter_score['mean_score']

0it [00:00, ?it/s]

word Pyrrhic is not in the dictionary.
word nester is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word flex is not in the dictionary.
word trice is not in the dictionary.
word pod is not in the dictionary.
word kilter is not in the dictionary.


0.6594036697247706

System | Score
--- | ---
Yandex | 0.41
Google | 0.49
OPUS | 0.65
mBART | 0.66

In [89]:
err_rate_df = pd.DataFrame(index=refs_df['idiom'].unique())
err_rate_df['yandex'] = ya_litter_score['per_idiom'].values()
err_rate_df['google'] = g_litter_score['per_idiom'].values()
err_rate_df['OPUS'] = opus_litter_score['per_idiom'].values()
err_rate_df['mBART'] = ya_litter_score['per_idiom'].values()
err_rate_df

,yandex,google,OPUS,mBART
a black sheep,0.00,0.00,0.00,0.00
a breath of fresh air,0.00,0.00,1.00,0.00
a dark horse,1.00,0.88,0.88,1.00
a fact of life,0.75,1.00,1.00,0.75
a fair-weather friend,0.75,1.00,0.25,0.75
...,...,...,...,...
when push comes to shove,1.00,1.00,1.00,1.00
work like a charm,0.00,1.00,1.00,0.00
wring your hands,1.00,1.00,1.00,1.00
you reap what you sow,0.83,0.83,0.50,0.83


In [90]:
err_rate_df.to_csv('/content/drive/MyDrive/data/idioms/idioms_error_rates.csv')